# Visualization of copy number distribution for human and mouse proteins

__Author:__ \
Emanuel Lange \
Mehrdimensionale OMICS-Datenanalyse \
ISAS e.V. \
Bunsen-Kirchhoff-Straße 11 \
44139 Dortmund, Germany \
emanuel.lange@isas.de

__Last Revision:__ \
September 29, 2023

__License:__

__Objective:__ \
This script generates interactive visualizations of protein copy number distributions for the study <insert manuscript name/id>.\

__How it works:__ \
For interactive visualization we utilized the bokeh library (http://bokeh.org/). \
Visualizations and data are stored as html documents and can be viewed using any modern web-browser.

__How to execute:__ \
Jupyter-notebooks are comprised of cells (text and code) that can be executed by CNTRL + ENTER (executes selected cell) or SHIFT + ENTER (executes selected cell and jumps to next one).\

Before running this script, make sure you installed and activated the provided CONDA environment (check our ANACONDA primer for more information). \
Furthermore, protein data spreadsheets (supplementary file <insert file ids\>) should be located in the root directory of this script.

## 1) Importing libraries

In [8]:
import pandas as pd #data handling
from bokeh.layouts import column, row, grid
from bokeh.plotting import figure, show # visualization
from bokeh.models import (ColumnDataSource, BoxAnnotation, DataTable, StringFormatter, NumberFormatter, TableColumn, Legend, LegendItem, CategoricalColorMapper, Circle, HoverTool, TextInput, CDSView, CustomJS, IndexFilter, Div) # utilities for visualization
from bokeh.palettes import BrBG4 # color palette for annotation boxes
from bokeh.io import output_notebook, output_file, export_svg  # output visualization in notebook

## 2) Importing data
- human/mouse protein data: file "copy_number_distribution_human_mouse.xlsx"
- human/mouse 10^5 protein data: file "copy_number_distribution_human_mouse_105_xlsx.xlsx"

In [21]:
human_data_105 = pd.read_excel(
    'copy_number_distribution_human_mouse_105.xlsx',
    sheet_name='Human')

mouse_data_105 = pd.read_excel(
    'copy_number_distribution_human_mouse_105.xlsx',
    sheet_name='Mouse')

In [3]:
human_data_105

,Rank,UniprotAccession,Descriptions,Copy number
0,1,P62805,Histone H4,1.602731e+08
1,2,P04908,Histone H2A type 1-B/E,5.074864e+07
2,3,P06899,Histone H2B type 1-J,4.422288e+07
3,4,P24158,Myeloblastin,2.943937e+07
4,5,P05109,Protein S100-A8,2.889194e+07
...,...,...,...,...
4192,4193,Q92508,Piezo-type mechanosensitive ion channel compon...,2.204672e+02
4193,4194,Q99996,A-kinase anchor protein 9,1.941876e+02
4194,4195,P49848,Transcription initiation factor TFIID subunit 6,1.818736e+02
4195,4196,P29374,AT-rich interactive domain-containing protein 4A,1.473175e+02


In [4]:
mouse_data_105

,Rank,UniprotAccession,Käsekuchen,Copy number
0,1,P10853,Histone H2B type 1-F/J/L;Histone H2B type 1-H;...,4.515535e+07
1,2,O08692,Neutrophilic granule protein,4.333381e+07
2,3,P31725,Protein S100-A9,3.958316e+07
3,4,P62806,Histone H4,3.681134e+07
4,5,P43274,Histone H1.4,2.888777e+07
...,...,...,...,...
5071,5072,Q6PDK2,Histone-lysine N-methyltransferase 2D,5.996033e+02
5072,5073,Q62504,Msx2-interacting protein,5.898587e+02
5073,5074,Q9R0G7,Zinc finger E-box-binding homeobox 2,5.850468e+02
5074,5075,A2AAE1,Transmembrane protein KIAA1109,5.335464e+02


Define function to generate plot

In [46]:
def make_data_source(data):
    data_dict = {}

    if not 'Rank' in data.columns or not 'Copy number' in data.columns:
        raise ValueError("Data must contain columns 'Rank' and 'Copy number'")

    for column in data.columns:

        data_dict[column] = data[column]

    source = ColumnDataSource(data_dict)

    return source

def make_data_table(data_source, view):
    columns = []

    for column in data_source.column_names:
        formatter = StringFormatter(font_style="bold")
        if (data_source.data[column].dtype == 'float64' or data_source.data[column].dtype == 'float32'):
            formatter = NumberFormatter(format='0.00',text_align='right')

        columns.append(TableColumn(field=column, title=column, formatter=formatter))

    return DataTable(
        source=data_source,
        view=view,
        columns=columns,
        editable=False,
        scroll_to_selection=True,
        margin=(20, 20, 20, 20), 
        sizing_mode="stretch_both",
        )

def initialize_figure(title, title_size):
        p = figure(
        title=title,
        x_axis_label="protein rank",
        y_axis_label="Log10(protein copy number)",
        y_axis_type="log",
        tools="pan,wheel_zoom,ybox_select,tap,reset,save",
        active_drag="ybox_select",
        margin=(20, 20, 20, 20),
        # lod_factor = 1000,
        # frame_width = 200,
        output_backend="webgl",
        sizing_mode="stretch_both",
        )
        p.title.align = 'center'
        p.title.text_font_size = str(title_size)+'pt'

        return p

def set_axes_and_grid(p, data_source, axis_label_size, tick_label_size):
    p.xaxis.bounds = (data_source.data['Rank'].min(), data_source.data['Rank'].max())
    p.yaxis.bounds = (data_source.data['Copy number'].min(), data_source.data['Copy number'].max())

    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    p.xaxis.axis_label_text_font_size = str(axis_label_size)+'pt'
    p.yaxis.axis_label_text_font_size = str(axis_label_size)+'pt'

    p.xaxis.major_label_text_font_size  = str(tick_label_size)+'pt'
    p.yaxis.major_label_text_font_size  = str(tick_label_size)+'pt'

def add_annotation_boxes(p, legend_font_size):
    ## annotation boxes
    high_abundances_box = BoxAnnotation(bottom=500000, fill_alpha=0.3, fill_color=BrBG4[3], level='underlay')
    low_abundances_box = BoxAnnotation(top=10000, fill_alpha=0.1, fill_color=BrBG4[0], level='underlay')
    p.add_layout(high_abundances_box)
    p.add_layout(low_abundances_box)

    ## legend
    r_high_abundance = p.square(fill_alpha=0.2, color=BrBG4[3])
    r_low_abundance = p.square(fill_alpha=0.2, color=BrBG4[0])
    r_human_proteins = p.circle(fill_alpha=0.4, color="dodgerblue")
    r_mouse_proteins = p.circle(fill_alpha=0.4, color="tomato")

    p.add_layout(Legend(items=[
        LegendItem(label='high abundance', renderers=[r_high_abundance], ),
        LegendItem(label='low abundance', renderers=[r_low_abundance]),
        LegendItem(label='human proteins', renderers=[r_human_proteins]),
        LegendItem(label='mouse proteins', renderers=[r_mouse_proteins]),
        ]
        ))

    p.legend.label_text_font_size = str(legend_font_size)+'pt'

def add_data_points(p, source):

    # add mapper
    mapper = CategoricalColorMapper(palette=["tomato", "dodgerblue"], factors=["mouse", "human"])

    # add circle renderer
    r_circle = p.circle(
        source=source,
        # view=view,
        x='Rank',
        y='Copy number',
        size=10,
        # color="navy",
        alpha=0.4,
        hover_alpha=1,
        # legend_label="proteins",
        color={'field': 'category', 'transform': mapper},
        # legend_field='category'
        level='glyph'
        )

    # define circle selection and nonselection properties
    selected_circle = Circle(fill_alpha=1, fill_color={'field': 'category', 'transform': mapper}, line_color=None)
    nonselected_circle = Circle(fill_alpha=0.5, fill_color="grey", line_color=None)

    r_circle.selection_glyph = selected_circle
    r_circle.nonselection_glyph = nonselected_circle

def add_hover(p, columns):
    ## hover tooltip for data points
    # custom tooltip layout

    hover_content = """<div @tooltip{custom}>"""

    for column in columns:
        hover_string = """    <b>{}</b>: @{{{}}} <br>""".format(column, column)
        hover_content += hover_string

    hover_content += """
    </div>
    <style>
        div.bk-tooltip-content > div > div:not(:first-child) {
            display:none !important;
            }
    </style>"""

    # initiate and add hover tool to display tooltips
    hover = HoverTool()
    hover.tooltips = hover_content
    p.add_tools(hover)

def make_plot(data_source, title, title_size, axis_label_size, tick_label_size, legend_font_size):
    p = initialize_figure(title, title_size)
    set_axes_and_grid(p, data_source, axis_label_size, tick_label_size)
    add_data_points(p, data_source)
    add_hover(p, data_source.column_names)
    add_annotation_boxes(p, legend_font_size)
    return p

## compose visualization
def make_widgets(data, source, filter):
    columns = list(data.columns)
    
    search_input = TextInput(value="", title="Search", placeholder="Type and hit Enter to search", width=300, margin=(20, 20, 10, 20))

    search_input.js_on_change(
        'value',
        CustomJS(
            args=dict(
                search_input=search_input,
                source=source,
                filter=filter,
                columns=columns),
            code="""
            function filterAll(obj, columns, val) {
                let stripped_val = val.replace(/[\s.;,\/_#]/g, '')
                stripped_val = stripped_val.replace('-', '')
                
                if (stripped_val.length === 0) {
                    return obj[columns[0]].map((entry, i) => i)
                }

                let indices = []

                for (let column of columns) {
                    indices = indices.concat(filterByColumn(obj, column, stripped_val))
                }

                return [...new Set(indices)]
            }

            function filterByColumn(obj, column, val) {

                const indices = []

                obj[column].forEach((entry, i) => {
                    let stripped_entry = String(entry).replace(/[\s.;,\/_#]/g, '')
                    stripped_entry = stripped_entry.replace('-', '')

                    if (String(stripped_entry).toLowerCase().includes(String(val).toLowerCase())) {
                        indices.push(i)
                    }
                })

                return indices
            }
            
            filter.indices = filterAll(source.data, columns, search_input.value)
            source.change.emit();
            """))
    
    template_total_proteins = ("""
                <div><b>Total number of proteins:</b> {total_proteins}</div>
                """)
    content_total_proteins = template_total_proteins.format(total_proteins=len(data))
    total_proteins_div = Div(text=content_total_proteins)

    selected_proteins_div = Div(text="<b>Selected proteins:</b> 0")
    
    source.selected.js_on_change(
        'indices',
        CustomJS(
            args=dict(div_selected=selected_proteins_div),
            code ="""
                // console.log(this.indices)
                div_selected.text = "<b>Selected proteins:</b> " + this.indices.length
            """
            )
            )
    
    template_filtered_proteins = ("""
                <div><b>Number of filtered proteins:</b> {filtered_proteins}</div>
                """)
    content_filtered_proteins = template_filtered_proteins.format(filtered_proteins=len(data))
    filtered_proteins_div = Div(text=content_filtered_proteins)

    source.js_on_change(
        "change",
        CustomJS(
            args=dict(filter=filter, filtered_proteins_div=filtered_proteins_div),
            code="""
                filtered_proteins_div.text = "<b>Number of filtered proteins:</b> " + filter.indices.length
            """
        )
    )

    return column(search_input, row(total_proteins_div, filtered_proteins_div, selected_proteins_div, margin=(0, 20, 10, 20)), sizing_mode='stretch_width')

def make_interactive_plot(
        data,
        title,
        output_file_name=None,
        title_size=20,
        axis_label_size=14,
        tick_label_size=14,
        legend_font_size=14):

    output_notebook()

    if output_file_name:
        output_file(filename=output_file_name+".html", title=title, mode="inline")

    data_source = make_data_source(data)
    filter = IndexFilter(list(range(len(data))))
    view = CDSView(filter=filter)

    data_table = make_data_table(data_source, view)

    plot = make_plot(data_source, title, title_size, axis_label_size, tick_label_size,legend_font_size)
    # plot.output_backend = "svg"

    widgets = make_widgets(data, data_source, filter)

    layout = row(column(widgets, data_table, sizing_mode="stretch_both"), column(plot, sizing_mode='stretch_both'), sizing_mode='stretch_both')
    show(layout)


## 3) Combined Visualization

merging human and mouse data

In [23]:
mouse_data_105['category'] = 'mouse'
human_data_105['category'] = 'human'

merged_data = pd.concat(
    [human_data_105, mouse_data_105],
    ignore_index=True,
    keys=['Human', 'Mouse'],)
merged_data

,Rank,UniprotAccession,Descriptions,Copy number,category
0,1,P62805,Histone H4,1.602731e+08,human
1,2,P04908,Histone H2A type 1-B/E,5.074864e+07,human
2,3,P06899,Histone H2B type 1-J,4.422288e+07,human
3,4,P24158,Myeloblastin,2.943937e+07,human
4,5,P05109,Protein S100-A8,2.889194e+07,human
...,...,...,...,...,...
9268,5072,Q6PDK2,Histone-lysine N-methyltransferase 2D,5.996033e+02,mouse
9269,5073,Q62504,Msx2-interacting protein,5.898587e+02,mouse
9270,5074,Q9R0G7,Zinc finger E-box-binding homeobox 2,5.850468e+02,mouse
9271,5075,A2AAE1,Transmembrane protein KIAA1109,5.335464e+02,mouse


In [47]:
title = "Human and Mouse_copy number per cell (10^5)"

make_interactive_plot(
    merged_data,
    title,
    output_file_name="human_mouse_105_copy_number_plot",
    title_size=16,
    axis_label_size=14,
    tick_label_size=14,
    legend_font_size=14
    )

Loading BokehJS ...